In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 데이터분석 4종 세트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

base_path = "/content/drive/MyDrive/Colab Notebooks/platform_subscribe/"

train = pd.read_csv(base_path + "train.csv")
test = pd.read_csv(base_path + "test.csv")
submission = pd.read_csv(base_path + "sample_submission.csv")

In [ ]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.7 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# 평가 지표
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

evulation_metric = f1_score

### second_best = ['average_time_per_learning_session', 'recent_learning_achievement'] + data_scaling + catboost

In [22]:
second_best = ['average_time_per_learning_session', 'recent_learning_achievement']
third_best = ['average_time_per_learning_session',
   'recent_learning_achievement',
   'average_login_time',
   'subscription_duration',
   'recent_login_time',
   'monthly_active_learning_days',
   'total_completed_courses',
   'abandoned_learning_sessions',
   'payment_pattern',
   'customer_inquiry_history']

In [23]:
X=train[second_best]
y=train.target
X

,average_time_per_learning_session,recent_learning_achievement
0,8.427187,68.360455
1,72.646087,97.567322
2,21.774492,94.358763
3,42.659066,70.153228
4,30.744287,81.917908
...,...,...
9995,84.053558,64.966803
9996,45.464833,82.750244
9997,127.302411,81.567839
9998,5.297234,89.885656


In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size=0.25, random_state=42)

In [25]:
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
std.fit(X_train)
X_train_scaled = std.transform(X_train)
X_val_scaled = std.transform(X_val)

In [26]:
clf_cb = CatBoostClassifier()
clf_cb.fit(X_train_scaled, y_train)
pred_train_cb=clf_cb.predict(X_train_scaled)
pred_val_cb=clf_cb.predict(X_val_scaled)

evulation_metric_train_cb=evulation_metric(y_train, pred_train_cb)
evulation_metric_val_cb=evulation_metric(y_val, pred_val_cb)
print(evulation_metric_train_cb)
print(evulation_metric_val_cb)

Learning rate set to 0.022141
0:	learn: 0.6901276	total: 2.96ms	remaining: 2.96s
1:	learn: 0.6877636	total: 5.8ms	remaining: 2.89s
2:	learn: 0.6850391	total: 8.46ms	remaining: 2.81s
3:	learn: 0.6825232	total: 10.9ms	remaining: 2.71s
4:	learn: 0.6802172	total: 13ms	remaining: 2.58s
5:	learn: 0.6779179	total: 15.1ms	remaining: 2.51s
6:	learn: 0.6761880	total: 17.4ms	remaining: 2.47s
7:	learn: 0.6738907	total: 19.8ms	remaining: 2.45s
8:	learn: 0.6722546	total: 22.3ms	remaining: 2.46s
9:	learn: 0.6708155	total: 24.6ms	remaining: 2.44s
10:	learn: 0.6699186	total: 27.1ms	remaining: 2.43s
11:	learn: 0.6688892	total: 29.3ms	remaining: 2.41s
12:	learn: 0.6679231	total: 31.9ms	remaining: 2.42s
13:	learn: 0.6669703	total: 34.2ms	remaining: 2.41s
14:	learn: 0.6659419	total: 36.5ms	remaining: 2.4s
15:	learn: 0.6645934	total: 39.1ms	remaining: 2.4s
16:	learn: 0.6635068	total: 42.5ms	remaining: 2.46s
17:	learn: 0.6620531	total: 47.5ms	remaining: 2.59s
18:	learn: 0.6613873	total: 54ms	remaining: 2.79s

### test 확인

In [27]:
X_test_scaled = std.transform(X_test)
X_test_scaled

array([[-0.73077477,  0.90464221],
       [-0.77659011, -0.61644965],
       [-0.51640995,  1.40444115],
       ...,
       [-0.81444743, -0.32212759],
       [-0.49334975, -0.03994693],
       [-0.13814892,  0.41189355]])

In [28]:
pred_test_cb=clf_cb.predict(X_test_scaled)

evulation_metric_test_cb=evulation_metric(y_test, pred_test_cb)
print(evulation_metric_test_cb)

0.7472668810289389
